In [1]:
def yolo_to_corners(label, img_width, img_height):
    """
    Преобразует лейбл YOLO в координаты углов прямоугольника.

    :param label: Лейбл YOLO в формате [class_id, x_center, y_center, width, height]
    :param img_width: Ширина изображения в пикселях
    :param img_height: Высота изображения в пикселях
    :return: Кортеж с координатами углов прямоугольника (x_min, y_min, x_max, y_max)
    """
    class_id, x_center_norm, y_center_norm, width_norm, height_norm = label

    # Денормализация координат+

    x_center = x_center_norm * img_width
    y_center = y_center_norm * img_height
    width = width_norm * img_width
    height = height_norm * img_height

    # Вычисление координат углов
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return x_min, y_min, x_max, y_max

In [ ]:
import os 
import cv2 as cv
import shutil

path_img_train = "F:/datasets/images/train/"
path_label_train = "F:/datasets/labels/train/"

path_img_test = "F:/datasets/images/test/"
path_label_test = "F:/datasets/labels/test/"

files_img = sorted(os.listdir(path_img_train))
files_label = sorted(os.listdir(path_img_train))


for i, filename in enumerate(files_img):
    img = cv.imread(path_img_train + filename)
    h, w = img.shape[:2]
    label_list = []

    with open(path_label_train + filename.replace(".jpg", ".txt"), 'r') as f:
        for line in f.readlines():
            label, x1, y1, x2, y2 = line.split()
            label_list.append([int(label), float(x1), float(y1), float(x2), float(y2)])

    
    for label in label_list:
        x_min, y_min, x_max, y_max = yolo_to_corners(label, w, h)
        print(x_min, y_min, x_max, y_max)
        cv.rectangle(img, (x_min, y_min),(x_max, y_max), (255,0,0),3)

    cv.imshow("image", img)


    while True:
        key = cv.waitKey(0) & 0xFF
        if key == ord('e'):
            print(f"Продолжение цикла. {i}")
            shutil.move(path_img_train + filename, "F:/datasets/images_corrent/train/" + filename)
            shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_corrent/train/" + filename.replace(".jpg", ".txt"))
            break
        elif key == ord('q'):  
            print(f"Неправильная разметка. {i}")
            shutil.move(path_img_train + filename, "F:/datasets/images_incorrent/train/" + filename)
            shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_incorrent/train/" + filename.replace(".jpg", ".txt"))
            break

cv.destroyAllWindows()

605 107 716 262
Продолжение цикла. 0
151 232 208 319
277 115 499 407
518 258 559 330
670 272 721 339
Продолжение цикла. 1
199 86 461 410
Продолжение цикла. 2
567 31 635 126
Продолжение цикла. 3
62 384 78 424
111 398 134 426
514 403 565 473
631 429 685 504
695 359 713 381
824 451 857 521
Продолжение цикла. 4
261 287 616 823
Продолжение цикла. 5
141 53 184 94
144 323 238 435
434 177 538 309
835 378 921 537
Продолжение цикла. 6
624 123 842 397
Продолжение цикла. 7
329 436 366 483
Продолжение цикла. 8
127 200 208 280
466 206 550 334
609 72 745 300
Продолжение цикла. 9
230 418 280 552
508 283 626 429
Продолжение цикла. 10
535 281 646 434
930 0 1001 140
Продолжение цикла. 11
408 82 567 272
Продолжение цикла. 12
373 129 430 192
691 171 749 230
896 126 961 205
964 120 1019 238
997 139 1023 272
Продолжение цикла. 13
445 333 624 535
Продолжение цикла. 14
339 117 453 305
629 132 739 312
Продолжение цикла. 15
209 207 341 383
525 105 685 319
845 177 909 291
869 361 1022 629
Продолжение цикла. 16
16